In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_auc_score, recall_score, classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame

In [36]:
# import libraries for visualizations

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (9, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [37]:
df = pd.read_excel (r'/Users/MaeganBoosten/Thesis_Project/Final/Original.xlsx')
print (df)

     duration  credit_amount  intallment_rate  residence_since       age  \
0    0.256953       1.078480         0.918477        -0.765977 -0.751642   
1   -0.240857      -0.237923         0.918477         1.046987  2.590552   
2    0.505858      -0.248556         0.024147         0.140505 -1.279356   
3    0.256953       1.168508        -0.870183        -0.765977 -1.279356   
4    2.248194       1.165673         0.024147        -0.765977 -1.015499   
..        ...            ...              ...              ...       ...   
369  1.252574       1.757239        -0.870183        -0.765977 -0.839594   
370 -0.738668      -0.877693         0.918477         1.046987  1.535122   
371 -0.738668      -0.912429         0.918477        -0.765977  0.919455   
372  0.256953      -0.728473         0.918477         1.046987  0.215835   
373 -0.738668      -0.615760        -1.764514        -1.672459 -1.103451   

     number_of_existcr  number_of_dependents  telephon   foreign  \
0             1.027

In [38]:
#split dataset

In [39]:
x, y = df.drop('False Positive', axis=1), df['False Positive']


In [40]:
# summarize class distribution
print(Counter(y))
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
X_over, y_over = oversample.fit_resample(x, y)
# summarize class distribution
print(Counter(y_over))

Counter({0: 270, 1: 104})
Counter({1: 270, 0: 270})


In [41]:
x_train, x_test, y_train, y_test= train_test_split(X_over,y_over, test_size=.5, random_state=42)

In [42]:
x_train.shape, x_test.shape

((270, 57), (270, 57))

In [43]:
#Number of False Positives in y_train
sum(y_train)

136

In [44]:
#Number of False Positives in y_test
sum(y_test)

134

In [45]:
#Baseline for evaluation
y.value_counts(normalize=True)

0    0.721925
1    0.278075
Name: False Positive, dtype: float64

In [46]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, recall_score, classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import metrics

In [47]:
def acc_val(TP, TN, FP, FN, data):    
    accuracy = (TP + TN) / float(TP + TN + FP + FN)
    recall = TP / float(FN + TP)
    specificity = TN / (TN + FP)
    false_positive_rate = FP / float(TN + FP)
    precision = TP / float(TP + FP)
    F1 = 2 * (precision * recall) / (precision + recall)
    print('The values for {} are:'.format(data))
    print('Accuracy: {} ,Recall: {} , Specificity: {} , False Positive Rate: {} , Precision: {}, F1 Score: {}'.format(accuracy, recall, specificity, false_positive_rate,
                                                                                                                   precision, F1))

In [48]:
#Model: DecisionTree
tree= DecisionTreeClassifier(random_state=42)
# fit model
tree.fit(x_train, y_train)
# predict
pred_test= tree.predict(x_test)

In [49]:
pred_test.shape, y_test.shape

((270,), (270,))

In [50]:
accuracy_score(y_test, pred_test)

0.8

In [51]:
confusion_matrix(y_test, tree.predict(x_test))

array([[ 99,  37],
       [ 17, 117]])

In [52]:
print(classification_report(y_test, tree.predict(x_test)))

              precision    recall  f1-score   support

           0       0.85      0.73      0.79       136
           1       0.76      0.87      0.81       134

    accuracy                           0.80       270
   macro avg       0.81      0.80      0.80       270
weighted avg       0.81      0.80      0.80       270



In [53]:
confusion = metrics.confusion_matrix(y_test, tree.predict(x_test))
print(confusion)
TP1 = confusion[1, 1]
TN1 = confusion[0, 0]
FP1 = confusion[0, 1]
FN1 = confusion[1, 0]
acc_val(TP1, TN1, FP1, FN1, 'Test Data')

[[ 99  37]
 [ 17 117]]
The values for Test Data are:
Accuracy: 0.8 ,Recall: 0.8731343283582089 , Specificity: 0.7279411764705882 , False Positive Rate: 0.27205882352941174 , Precision: 0.7597402597402597, F1 Score: 0.8124999999999999


In [54]:
roc_auc_score(y_test, tree.predict(x_test))

0.8005377524143986

In [55]:
#Tuning using gridsearch

In [56]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, recall_score, classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate

In [57]:
params= {'max_depth': np.arange(2,11), 'min_samples_leaf': np.arange(2,11)  }

In [58]:
skf= StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [59]:
best_tree= GridSearchCV(tree, param_grid= params, cv=skf, n_jobs=-1, verbose=1)
best_tree

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'min_samples_leaf': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])},
             verbose=1)

In [60]:
params= {'max_depth': np.arange(2,11), 'min_samples_leaf': np.arange(2,11)  }
best_tree= GridSearchCV(tree, param_grid= params, cv=skf, n_jobs=-1, verbose=1)

In [61]:
best_tree.fit(x_train, y_train)


Fitting 10 folds for each of 81 candidates, totalling 810 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'min_samples_leaf': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])},
             verbose=1)

In [62]:
best_tree.best_params_


{'max_depth': 10, 'min_samples_leaf': 3}

In [63]:
best_tree.best_estimator_


DecisionTreeClassifier(max_depth=10, min_samples_leaf=3, random_state=42)

In [64]:
# cross-val assessment of model quality
best_tree.best_score_


0.7296296296296296

In [65]:
pred_test_better= best_tree.predict(x_test)

In [66]:
accuracy_score(y_test, best_tree.predict(x_test))

0.6962962962962963

In [67]:
confusion = metrics.confusion_matrix(y_test, best_tree.predict(x_test))
print(confusion)
TP1 = confusion[1, 1]
TN1 = confusion[0, 0]
FP1 = confusion[0, 1]
FN1 = confusion[1, 0]
acc_val(TP1, TN1, FP1, FN1, 'Test Data')

[[92 44]
 [38 96]]
The values for Test Data are:
Accuracy: 0.6962962962962963 ,Recall: 0.7164179104477612 , Specificity: 0.6764705882352942 , False Positive Rate: 0.3235294117647059 , Precision: 0.6857142857142857, F1 Score: 0.7007299270072992


In [68]:
roc_auc_score(y_test, best_tree.predict(x_test))

0.6964442493415277